In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,
classification_report
!pip install vecstack
from vecstack import stacking

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19865 sha256=6a8de3f8969d6fb784d5098203fd6c7970286a22eaefe7937fab9843733f2b55
  Stored in directory: /root/.cache/pip/wheels/b8/d8/51/3cf39adf22c522b0a91dc2208db4e9de4d2d9d171683596220
Successfully built vecstack


In [2]:
df = pd.read_excel("Social_Network_Ads.xlsx")
df = pd.get_dummies(df, drop_first=True)
df.head()

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1


In [3]:
df = df.drop("User ID",axis=1)

In [4]:
x = df.drop('Purchased',axis=1)
y = df["Purchased"]

In [5]:
x

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                    random_state= 10)

In [7]:
X_train.shape

(320, 3)

In [9]:
#Hyperparameter tuning for decision tree classifier
from sklearn.model_selection import RandomizedSearchCV
clf = DecisionTreeClassifier()
parameters={'min_samples_split' : np.arange(10,100,20),
'max_depth': np.arange(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15 ,scoring = 'roc_auc', 
                                cv =5 ,verbose = True)
clf_random.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=15,
                   param_distributions={'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([10, 30, 50, 70, 90])},
                   scoring='roc_auc', verbose=True)

In [13]:
import sklearn
ds_pred=clf_random.predict(X_test)
sklearn.metrics.roc_auc_score(y_test,ds_pred)

0.9340659340659341

In [14]:
{'min_samples_split': 70, 'max_depth': 9}

{'min_samples_split': 70, 'max_depth': 9}

In [15]:
rf = RandomForestClassifier(criterion='gini', max_depth=4,
                            max_features="auto", n_estimators=500)
rf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=4, max_features='auto', n_estimators=500)

In [17]:
rf_pred=rf.predict(X_test)
sklearn.metrics.roc_auc_score(y_test, rf_pred)

0.9162087912087912

In [19]:
knn = KNeighborsClassifier(weights='distance', n_neighbors= 7)
knn.fit(X_train , y_train)
knn_pred=knn.predict(X_test)
sklearn.metrics.roc_auc_score(y_test, knn_pred)

0.7994505494505495

In [20]:
from sklearn.svm import LinearSVC
svc=LinearSVC(max_iter= 700)
svc.fit(X_train , y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(max_iter=700)

In [21]:
svc_pred=svc.predict(X_test)
sklearn.metrics.roc_auc_score(y_test, svc_pred)

0.5

In [22]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

In [23]:
models = [KNeighborsClassifier(n_neighbors= 3) ,
DecisionTreeClassifier(min_samples_split= 70,max_depth=9),
RandomForestClassifier(min_samples_split= 90,max_depth=9),
MLPClassifier(max_iter= 100, learning_rate='constant',
hidden_layer_sizes= (20, 7, 3),
activation= 'tanh') ,
LinearSVC(max_iter= 700) ]
S_Train, S_Test = stacking(models,
X_train, y_train ,X_test ,
regression=False,
mode='oof_pred_bag',#out of fold data
needs_proba=False,
save_dir=None,
metric= roc_auc_score,
n_folds=5,
stratified=True,
shuffle=True,
random_state=0,
verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [roc_auc_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.78685048]
    fold  1:  [0.74602333]
    fold  2:  [0.75980912]
    fold  3:  [0.66595970]
    fold  4:  [0.84252386]
    ----
    MEAN:     [0.76023330] + [0.05756705]
    FULL:     [0.76023330]

model  1:     [DecisionTreeClassifier]
    fold  0:  [0.87115589]
    fold  1:  [0.91993637]
    fold  2:  [0.84676564]
    fold  3:  [0.92258749]
    fold  4:  [0.97560976]
    ----
    MEAN:     [0.90721103] + [0.04478923]
    FULL:     [0.90721103]

model  2:     [RandomForestClassifier]


/usr/local/lib/python3.10/dist-packages/vecstack/core.py:615: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  S_test[:, model_counter] = st.mode(S_test_temp, axis = 1)[0].ravel()
/usr/local/lib/python3.10/dist-packages/vecstack/core.py:615: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

    fold  0:  [0.87115589]
    fold  1:  [0.85471898]
    fold  2:  [0.87115589]
    fold  3:  [0.79215270]
    fold  4:  [0.91039236]


/usr/local/lib/python3.10/dist-packages/vecstack/core.py:615: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  S_test[:, model_counter] = st.mode(S_test_temp, axis = 1)[0].ravel()


    ----
    MEAN:     [0.85991516] + [0.03852118]
    FULL:     [0.85991516]

model  3:     [MLPClassifier]
    fold  0:  [0.50000000]
    fold  1:  [0.50000000]
    fold  2:  [0.50000000]
    fold  3:  [0.50000000]
    fold  4:  [0.50000000]
    ----
    MEAN:     [0.50000000] + [0.00000000]
    FULL:     [0.50000000]

model  4:     [LinearSVC]
    fold  0:  [0.50000000]
    fold  1:  [0.50000000]
    fold  2:  [0.50000000]
    fold  3:  [0.50000000]
    fold  4:  [0.50000000]
    ----
    MEAN:     [0.50000000] + [0.00000000]
    FULL:     [0.50000000]



/usr/local/lib/python3.10/dist-packages/vecstack/core.py:615: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  S_test[:, model_counter] = st.mode(S_test_temp, axis = 1)[0].ravel()
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, 

In [24]:
S_Test.shape

(80, 5)

In [25]:
mlp = MLPClassifier()
parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)],
'activation':['tanh', 'relu'], 'learning_rate':['constant',
'adaptive'], 'max_iter' :[1000, 150]}
mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15 ,
scoring = 'roc_auc' , cv =5 , verbose = True)
mlp_random.fit(S_Train , y_train)
grid_parm=mlp_random.best_params_
print(grid_parm)
print(mlp_random.best_score_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

{'max_iter': 1000, 'learning_rate': 'constant', 'hidden_layer_sizes': (20, 7, 3), 'activation': 'tanh'}
0.9235418875927891


In [26]:
mlp1 = MLPClassifier(max_iter= 150,learning_rate='adaptive',
hidden_layer_sizes=(10, 5, 3), activation= 'tanh')

In [27]:
mlp1.fit(S_Train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 5, 3),
              learning_rate='adaptive', max_iter=150)

In [28]:
pred1=mlp1.predict(S_Train)

In [29]:
y_train=np.array(y_train)

In [30]:
np.sum(pred1!=y_train)

31

In [31]:
y_train.shape

(320,)

In [32]:
pred1.shape

(320,)

In [33]:
mlp.fit(S_Train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [34]:
sklearn.metrics.roc_auc_score(y_train, mlp.predict(S_Train))

0.9072110286320254

In [35]:
pred2=mlp1.predict(S_Test)

In [37]:
sklearn.metrics.roc_auc_score(y_test, pred2)

0.9258241758241759